# Convolutional Neural Netrowk Model - DEAP dataset, Valence emotion 
# (Three Classes)

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras import metrics
from keras import regularizers
import models.nnmodels as nn
import utils.data as data

import gc
gc.enable()
seed = 0
np.random.seed(seed)
%matplotlib inline

Using TensorFlow backend.


# Load data for three classes classification


In [2]:
print 'Loading DEAP dataset...'
subject_data_folds, valence_folds, arousal_folds = data.load_deap(32, 3)
print 'DEAP dataset is loaded.\n'

print 5*'#' + ' Dataset representation for each subject ' + 5*'#'
print '\nThere are {} subjects in the dataset.\n'.format(len(subject_data_folds))
print '(Trail/Video, Channel, Readings) : ', subject_data_folds[0].shape
print '(Trail/Video, label (valence or arousal)) : ', arousal_folds[0].shape



Loading DEAP dataset...
DEAP dataset is loaded.

##### Dataset representation for each subject #####

There are 32 subjects in the dataset.

(Trail/Video, Channel, Readings) :  (40, 40, 8064)
(Trail/Video, label (valence or arousal)) :  (40, 3)


# Reduce the features dimention (8064D >> 101D)
## Then Normalize the features

In [3]:
# Reduce the features dimension
print 'Mapping features into Gaussian with mean zero and unit variance...'
reduced_data_folds = data.normalize_features(subject_data_folds, flag=True)
print 'Normalization done.'

Mapping features into Gaussian with mean zero and unit variance...
Normalization done.


# Train CNN model for three classes (low Valence / mid Valence / High Valence) classification

In [4]:
num_exp = 40
num_classes = 3
batch_size = 50
epochs = 250
num_subjects = 32


accuracies= []

print '##### Three Classes Classification for Valence emotion Using Convolutional Neural Network #####\n'
   

for s in np.arange(0, num_subjects):
    
    
    X_val = reduced_data_folds[s]
    X_val = X_val.reshape(X_val.shape[0], 1, 40, 101)
    y_val = valence_folds[s]
    
    X_remaining_folds = reduced_data_folds[:s] + reduced_data_folds[s+1:]
    y_remaining_folds = valence_folds[:s] + valence_folds[s+1:]
    
    X_train = np.concatenate(X_remaining_folds)
    X_train = X_train.reshape(X_train.shape[0], 1, 40, 101)
    
    y_train = np.concatenate(y_remaining_folds)
    
    
    key = 's'+str(s+1).zfill(2)+'_valence(3classes)'
    
    
    # Valence classification model for three classes    
    model = nn.CNN(num_classes=num_classes, emotion='valence')
    
    
    # checkpoint
    filepath="../saved_models/best_CNN_model_for_{}.h5".format(key)
    
    print 'Training model to test on subject {} ...\n'.format(str(s+1).zfill(2))
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                verbose=1, save_best_only=True, mode='max', period=20)
    
    callbacks_list = [checkpoint]
    
    
    hist = model.fit(X_train, y_train, 
             batch_size=batch_size,
             epochs=epochs,
             shuffle=True,
             verbose=0,
             validation_data=(X_val, y_val), callbacks=callbacks_list)

    
    
    model.load_weights(filepath)
    
    scores = model.evaluate(x=X_val, y=y_val)
    
    
    
    print '#### Result of CNN model for subject {} #####\n'.format(str(s+1).zfill(2))
    print "Loss: {} , Accuracy : {}%".format(scores[0], scores[1]*100)
    
    

    accuracies.append(scores[1])
    # summarize history for accuracy
    plt.plot(hist.history['categorical_accuracy'])
    plt.plot(hist.history['val_categorical_accuracy'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model accuracy for Valence')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model loss for Valence')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    del model
 

   

##### Three Classes Classification for Valence emotion Using Convolutional Neural Network #####



ValueError: The channel dimension of the inputs should be defined. Found `None`.

# Accuracies

In [ ]:
min_acc_subject_idx, max_acc_subject_idx = np.argmin(accuracies), np.argmax(accuracies)

avg_acc =  reduce(lambda x, y: x + y, accuracies) / len(accuracies)

print 5*'#' + ' Accuracies per subject ' + 5*'#'

print 'Subject {} has the minimum accuracy : {}%'.format(min_acc_subject_idx + 1, min(accuracies)*100)
print 'Subject {} has the maximum accuracy : {}%'.format(max_acc_subject_idx + 1, max(accuracies)*100)
print 'Average accuracy : {}%'.format(avg_acc*100)

for subj in enumerate(accuracies):
    print 'Accuracy of Subject {} : {}%'.format(subj[0]+1, subj[1]*100)